In [85]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [86]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy
import warnings 
warnings.filterwarnings('ignore')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (146 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
import seaborn as sns
import numpy as np
import pandas as pd
import re
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Kkma
kkma = Kkma()

In [88]:
movie = pd.read_csv('/content/gdrive/My Drive/data/movies.csv')
movie.drop(['Unnamed: 0'], axis = 1, inplace=True)
movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink
0,포드 V 페라리,액션,관람객 평점 9.31점,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...
1,감쪽같은 그녀,드라마,관람객 평점 8.83점,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...
2,백두산,드라마,관람객 평점 7.47점,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...
3,시동,드라마,관람객 평점 8.68점,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...
4,블랙머니,범죄,관람객 평점 8.75점,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...


결측값 확인

In [89]:
movie.isnull().sum()

title          0
genre          0
vote          77
vote_count     0
director       0
cast           0
summary        0
imagelink      0
dtype: int64

평점이 null인 행만 추출

In [90]:
movie[movie['vote'].isnull()][:5]

,title,genre,vote,vote_count,director,cast,summary,imagelink
49,이터널스,액션,NaN,0,클로이 자오,"['안젤리나 졸리', '마동석', '리차드 매든']","잭 커비의 코믹북을 원작으로, 수백만 년 전 인류를 실험하기 위해 지구로 온 셀레스...",https://movie-phinf.pstatic.net/20190813_248/1...
63,미친사랑,드라마,NaN,0,문시현,"['공형진', '이시원', '송재림']",과거 북파 공작원으로 활동했던 남자(두식)는 고향(남한)으로도 돌아가지 못한 채 수...,https://movie-phinf.pstatic.net/20190530_108/1...
67,아이언맨,SF,NaN,0,존 파브로,"['로버트 다우니 주니어', '테렌스 하워드', '제프 브리지스']",하이테크 슈퍼 히어로의 탄생 | 이제 업그레이드는 끝났다천재적인 두뇌와 재능으로 세...,https://movie-phinf.pstatic.net/20111222_37/13...
70,헬보이 2: 골든 아미,SF,NaN,0,길예르모 델 토로,"['론 펄먼', '셀마 블레어', '더그 존스']",잘생긴 얼굴만 세상을 구하는 건 아니지!인간과 요괴 사이에서 존재하던 고대의 휴전 ...,https://movie-phinf.pstatic.net/20111222_198/1...
74,해리 포터와 비밀의 방,판타지,NaN,0,크리스 콜럼버스,"['다니엘 래드클리프', '루퍼트 그린트', '엠마 왓슨']",해리 포터에겐 이번 여름방학이 별로 즐겁질 못했다. 마법이라면 질색을 하는 페투니아...,https://movie-phinf.pstatic.net/20111222_244/1...


vote가 null인 값들을 직접 naver 영화에서 검색해보았다. 
아직까지 개봉하지 않은 영화이거나, 개봉했으나 관람객 평점이 존재하지 않는 경우이므로 추천하는데에 무의미한 데이터라 생각하고 drop 하여 삭제하는 것으로 결정하였다.

In [91]:
movie = movie.dropna()
movie.isnull().sum()

title         0
genre         0
vote          0
vote_count    0
director      0
cast          0
summary       0
imagelink     0
dtype: int64

In [92]:
movie.reset_index(inplace = True)
movie.drop(['index'], axis=1, inplace = True)

줄거리를 형태소 분리하여 띄워쓰기로 구분한다.

In [93]:
include = ['NNG','NP','NNP','NNB','NN','NX','UM','NU','VV','VA','AJ','IC']

명사와 동사, 형용사만 포함될 수 있도록 하며 한 글자로 된 단어는 제외했다. 

In [94]:
movie['summary_processed'] = ''
for i in range(len(movie)):
  pos = kkma.pos(movie['summary'][i])
  res = set()
  for j in range(len(pos)):
    if pos[j][1] in include:
      if len(pos[j][0]) > 1:
        res.add(pos[j][0])
  movie['summary_processed'][i] = ' '.join(res)

movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed
0,포드 V 페라리,액션,관람객 평점 9.31점,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,질주 판매 즐기 열정 대결 헨리 박살 만들 대회 만큼 실력 여기 입맛 합병 펼치 지...
1,감쪽같은 그녀,드라마,관람객 평점 8.83점,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,생활 혼자 지내 서로 즐기 특기 되어가 실례 손녀 자수 주름 필요 와의 나타나 취향...
2,백두산,드라마,관람객 평점 7.47점,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,운명 일급 접선 무력 초유 따르 만들 순식간 행동 지질학 예측 펼쳐지 곤란 계획 한...
3,시동,드라마,관람객 평점 8.68점,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,학교 상상 택일 거석 염정 반점 인사 사회 세상 나누 뛰어들 무섭 인생 맛보 동석 ...
4,블랙머니,범죄,관람객 평점 8.75점,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,은행 자산가 내몰리 직진 회사 이름 검찰 대한 프로 자살 금융 거대 지검 아침 비리...


vote를 평점만 남긴 데이터로 전처리한다.

In [95]:
for i in range(len(movie)):
  vote = movie['vote'][i]
  movie['vote'][i] = float(vote[6:len(vote)-1])

movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed
0,포드 V 페라리,액션,9.31,"1,118",제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,질주 판매 즐기 열정 대결 헨리 박살 만들 대회 만큼 실력 여기 입맛 합병 펼치 지...
1,감쪽같은 그녀,드라마,8.83,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,생활 혼자 지내 서로 즐기 특기 되어가 실례 손녀 자수 주름 필요 와의 나타나 취향...
2,백두산,드라마,7.47,"2,828",이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,운명 일급 접선 무력 초유 따르 만들 순식간 행동 지질학 예측 펼쳐지 곤란 계획 한...
3,시동,드라마,8.68,"1,780",최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,학교 상상 택일 거석 염정 반점 인사 사회 세상 나누 뛰어들 무섭 인생 맛보 동석 ...
4,블랙머니,범죄,8.75,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,은행 자산가 내몰리 직진 회사 이름 검찰 대한 프로 자살 금융 거대 지검 아침 비리...


vote_count 또한 int형으로 바꾸어준다..

In [96]:
for i in range(len(movie)):
  if type(movie['vote_count'][i]) != int and ',' in movie['vote_count'][i]:
    a = list(movie['vote_count'][i])
    a.pop(a.index(','))
    a = ''.join(a)
    movie['vote_count'][i] = int(a)
  else:
    movie['vote_count'][i] = int(movie['vote_count'][i])

가중 평점 = (v/(v+m)) R + (m/(v+m)) \ C

* v: 개별 영화에 평점을 투표한 횟수
* m: 평점을 부여하기 위한 최소 투표 횟수
* R: 개별 영화에 대한 평균 평점
* C: 전체 영화에 대한 평균 평점


v 는 'vote_count' 값이며, R은 'vote_average' 값에 해당합니다. C의 경우 전체 영화의 평균 평점이므로 movies['ratings'].mean()으로 구할 수 있습니다.
m 의 경우는 투표 횟수에 따른 가중치를 직접 조절하는 역할을 하는데, m을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 주게 됩니다.
여기서는 m을 상위 60% 해당하는 횟수를 기준으로 정하겠습니다.

In [97]:
C = movie['vote'].mean()
m = movie['vote_count'].quantile(0.6)
print('C:',round(C,3),'m:',round(m,3))

C: 8.465 m: 312.4


In [98]:
percentile = 0.6
C = movie['vote'].mean()
m = movie['vote_count'].quantile(0.6)

def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote']

  return ((v/(v+m)) * R) +( (m/(v+m)) * C)

movie['weighted_vote'] = movie.apply(weighted_vote_average, axis = 1)
movie.head(5)

,title,genre,vote,vote_count,director,cast,summary,imagelink,summary_processed,weighted_vote
0,포드 V 페라리,액션,9.31,1118,제임스 맨골드,"['맷 데이먼', '크리스찬 베일']","자존심을 건 대결의 시작!1960년대, 매출 감소에 빠진 ‘포드’는 판매 활로를 찾...",https://movie-phinf.pstatic.net/20191204_91/15...,질주 판매 즐기 열정 대결 헨리 박살 만들 대회 만큼 실력 여기 입맛 합병 펼치 지...,9.125478
1,감쪽같은 그녀,드라마,8.83,208,허인무,"['나문희', '김수안']",“초면에 실례하겠습니다~”\n 72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생...,https://movie-phinf.pstatic.net/20191204_6/157...,생활 혼자 지내 서로 즐기 특기 되어가 실례 손녀 자수 주름 필요 와의 나타나 취향...,8.610960
2,백두산,드라마,7.47,2828,이해준,"['이병헌', '하정우', '마동석']",대한민국 관측 역사상 최대 규모의 백두산 폭발 발생.\n 갑작스러운 재난에 한반도는...,https://movie-phinf.pstatic.net/20191219_263/1...,운명 일급 접선 무력 초유 따르 만들 순식간 행동 지질학 예측 펼쳐지 곤란 계획 한...,7.568992
3,시동,드라마,8.68,1780,최정열,"['마동석', '박정민', '정해인']",학교도 싫고 집도 싫고 공부는 더더욱 싫다며\n '엄마'(염정아)에게 1일 1강스파...,https://movie-phinf.pstatic.net/20200117_56/15...,학교 상상 택일 거석 염정 반점 인사 사회 세상 나누 뛰어들 무섭 인생 맛보 동석 ...,8.647918
4,블랙머니,범죄,8.75,862,정지영,"['조진웅', '이하늬']",고발은 의무! 수사는 직진!\n할말은 하고 깔 건 깐다!일명 서울지검 ‘막프로’! ...,https://movie-phinf.pstatic.net/20191113_203/1...,은행 자산가 내몰리 직진 회사 이름 검찰 대한 프로 자살 금융 거대 지검 아침 비리...,8.674220


* 가중 평점을 적용하지 않은 평점 순위 Top 10

In [99]:
movie[['title','vote','weighted_vote','vote_count']].sort_values('vote', ascending=False)[:10]

,title,vote,weighted_vote,vote_count
101,러브 앳,10,8.479719,3
186,앨리스 죽이기,10,8.474884,2
262,체 게바라: 2부 게릴라,10,8.470017,1
224,집으로...,10,8.498758,7
273,쿵푸 보이,10,8.479719,3
194,발리: 천상의 울림,10,8.470017,1
287,너의 목소리,10,8.474884,2
260,퍼펙트 타겟,10,8.470017,1
191,물의 기억,10,8.479719,3
7,사선의 끝,10,8.474884,2


* 가중 평점을 적용한 Top 10

In [100]:
movie[['title','vote','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote,weighted_vote,vote_count
326,어벤져스: 엔드게임,9.5,9.490055,32196
38,보헤미안 랩소디,9.45,9.427144,13149
34,그린 북,9.55,9.407861,2072
50,알라딘,9.42,9.386280,8534
37,가버나움,9.54,9.343216,1394
31,항거:유관순 이야기,9.39,9.254694,1823
231,스파이더맨: 뉴 유니버스,9.38,9.247214,1840
315,위대한 쇼맨,9.31,9.247133,3886
283,뽀로로 극장판 보물섬 대모험,9.35,9.223276,1869
217,극한직업,9.2,9.185950,16028


In [101]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
summ_mat = count_vect.fit_transform(movie['summary_processed'])
print(summ_mat.shape)

(334, 14746)


In [102]:
summ_sim = cosine_similarity(summ_mat, summ_mat)
summ_sim_sorted_ind = summ_sim.argsort()[:,::-1] #유사도가 높은 순으로 정리된 summ_sim 객체의 비교 행 위치 값을 얻을 수 있습니다.

추출해냈던 `summary_processed`에 대해 코사인 유사도를 구하여 줄거리 키워드가 유사한 영화 상위 10개를 추천해보자

In [103]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
    title_movie = df[df['title'] == title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

In [104]:
movie[movie['title'] == '어벤져스: 엔드게임'][['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
326,어벤져스: 엔드게임,9.5,9.490055,운명 펼쳐지 이후 위대 살아남 지구 떠나 위하 전쟁 절반 마지막 바꾸 최후 희망


In [105]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '어벤져스: 엔드게임', 10)
similar_movies[['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
314,겨울왕국,9.2,9.178613,서로 친구 환상적 언니 세상 최고 저주 동생 자매 떠나 위하 자신 신비 얼리 녹이 ...
59,토이 스토리 4,9.23,9.151045,운명 한편 거부 통하 새롭 모험 떠나 친구 위하 나서 여행 장난감 만나 세상 버즈 ...
64,어벤져스: 인피니티 워,9.08,9.067384,운명 펼쳐지 목격 걸리 클라이맥스 최강 무한 대결 이루 향하 마블 스톤 조합 세계 역대
293,드래곤볼 슈퍼: 브로리,9.47,8.847628,상상 운명 프리 한계 대회 만들 처절 엄청나 전투력 평화 수수께끼 전투 되찾 시작 ...
240,다크 나이트,9.65,8.617197,운명 기사 조직 결전 정체 전체 대결 매이 죽이 혼돈 펼치 부패 범죄 시작 내던지 ...
158,존 윅 3: 파라벨룸,8.64,8.604717,직면 킬러 파문 피아 내려지 전쟁 평화 국제 조치 찾아가 보호 세계 향하 마지막 결...
254,아일라,9.13,8.586829,운명 서로 잠시 무엇 이름 전쟁 터키어 참전 돌아가 발견 고국 그리움 모두 파병 지...
328,터미네이터 2,9.51,8.581085,운명 기계 인간 미래 전쟁 과거 액체 없애 사령관 지키 저항 위하 보내 스카이 코너...
327,터미네이터 2:오리지널,9.51,8.581085,운명 기계 인간 미래 전쟁 과거 액체 없애 사령관 지키 저항 위하 보내 스카이 코너...
278,"안녕, 티라노: 영원히, 함께",8.6,8.511366,이제 혼자 펼쳐지 우정 비밀 관하 사랑 약속 이야기 떠나 희망 니야 가슴 아프 간직 천국


In [106]:
movie[movie['title'] == '보헤미안 랩소디'][['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
38,보헤미안 랩소디,9.45,9.427144,아웃사이더 성장 퍼포먼스 이민자 독창적 들어가 데뷔 멤버 이름 결별 사로잡 시대 흔...


In [107]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '보헤미안 랩소디', 10)
similar_movies[['title','vote','weighted_vote','summary_processed']]

,title,vote,weighted_vote,summary_processed
231,스파이더맨: 뉴 유니버스,9.38,9.247214,싸우 이후 피터 거미 누아르 모든 시대 구하 유니 시작 버스 있음 존재 악당 마주치...
182,브링 더 소울 : 더 무비,9.68,9.074111,전세계 풀어지 파리 긴장 둘르 멤버 설레 콘서트 나누 펼쳐지 유럽 투어 시작 열광 ...
272,너의 이름은.,9.02,9.004857,운명 서로 친구 가족 깨닫 되어가 이름 확인 시작 다가오 도쿄 낯설 너르 남기 풍경...
133,마이펫의 이중생활2,9,8.854875,어로 스노우 집구석 따르 생각 걱정 세상 시작 비우 고프 자유 나쁘 속마음 공개 사...
145,범블비,8.75,8.706361,목격 서로 인간 의하 정체 임무 이름 전쟁 되어가 도착 지속 거대 발견 진짜 수리 ...
202,위로공단,9.02,8.657658,단지 신음 눈물 독창적 빌딩 노동 가족 이름 과거 기나길 동의 다큐멘터리 나이키 스...
254,아일라,9.13,8.586829,운명 서로 잠시 무엇 이름 전쟁 터키어 참전 돌아가 발견 고국 그리움 모두 파병 지...
21,로켓맨,8.8,8.585797,알아보 퍼포먼스 대중 자리잡 환상적 친구 상처 사로잡 시대 세상 음악성 최고 천재적...
136,명탐정 피카츄,8.53,8.508121,탐정 아냐 여기 이름 구하 적히 세상 깨어나 시작 단서 모험 아빠 번쩍이 아들 해리...
197,1919 유관순,8.65,8.477798,시작 기록 그녀 기억 위하 그날 맞서 이름 재판 남기 너희 시절 소녀 모든 세상 우...


줄거리에 나타나는 키워드 유사도를 바탕으로 추천해보았을 때엔 `나름대로..` 비슷한 키워드가 포함된 영화들이 추천되는 것 같다. 물론 엄청난 관련은 없어보이지만... 

이제는 좀 더 세부적인 조건을 추가해보려고 한다.
* 장르가 같으면 가산점
* 감독이 같으면 가산점
* 출연하는 배우가 같으면 가산점

---

### 장르가 같으면 가산점
장르는 기본적으로 포함하여 줄거리 유사도와 장르의 일치여부를 판단하여 추천해보려고 한다. 

### 감독, 출연하는 배우가 같으면 가산점
이 부분은 `선택`으로 줄 생각이다. 만약 입력되지 않는다면 장르와 줄거리 유사도를 기반으로 10가지 영화를 추천하는 반면에, 감독이나 배우가 입력된다면 유사도의 우선순위를 `장르 > 감독 or 배우 > 줄거리유사도` 순으로 주고, 입력되지 않는다면 유사도 우선순위를 `장르 > 줄거리유사도` 순으로 준다.


이는 해당 영화를 좋아한다고 해서 그 영화의 `감독`을 좋아하진 않는다는 가정을 바탕으로 진행하며 `배우` 또한 마찬가지이다. 

앞서 구현했던 find_sim_movie를 장르를 포함할 수 있도록 수정해보자.
대신, 동일한 장르의 영화만 10가지 모두 추천되지 않도록 같은 장르의 영화는 50% 추천될 수 있도록 해보자.

In [180]:
from sklearn.feature_extraction.text import CountVectorizer
def get_similarity(df):
  count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
  mat = count_vect.fit_transform(df['summary_processed'])

  sim = cosine_similarity(mat, mat)
  sim_sorted_ind = sim.argsort()[:,::-1]
  return mat, sim, sim_sorted_ind

In [174]:
summ_mat, summ_sim, summ_sim_sorted_ind = get_similarity('summary_processed', movie)

1. 장르가 일치하는 것에서 줄거리 유사도를 산충해서 높은 것 5가지를 뽑는다.
2. 줄거리 유사도가 일치하는 것들 중 1과 겹치지 않는 것을 추가한다.

In [227]:
def find_sim_movie(df, summ_sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    #먼저, 장르가 일치하는 것들에 대한 dataframe을 구한다.
    genre = df[df['title'] == title_name]['genre']
    genre_name = genre.values[0]

    print("추천 기준 영화 정보")
    print(df[df['title'] == title_name][['title','genre','summary_processed']])

    genre_df = df[df['genre'] == genre_name]
    genre_df.reset_index(inplace = True)
    genre_df.drop(['index'], axis=1, inplace = True)
    
    g_mat, g_sim, g_sim_sorted_ind = get_similarity(genre_df)
    g_title_movie = genre_df[genre_df['title'] == title_name]
    g_title_index = g_title_movie.index.values
    g_similar_indexes = g_sim_sorted_ind[g_title_index, :(top_n)]
    g_similar_indexes = g_similar_indexes.reshape(-1)
    g_similar_indexes = g_similar_indexes[g_similar_indexes != g_title_index]
    g = genre_df.iloc[g_similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

# /    줄거리가 유사한 것들에 대해 다시 진행
    similar_indexes = summ_sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경
    similar_indexes = similar_indexes[similar_indexes != title_index]
    s = df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

    # 혹시 모를 후보군
    candidate = df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n//2]

    result = pd.concat([g, s])
    result = result.sort_values('weighted_vote', ascending=False)
    result.reset_index(inplace = True)
    result.drop(['index'],axis=1,inplace=True)
    result = result.drop_duplicates(['title'], keep='first') #중복 제거

    if len(result) < 10: #열개보다 적은 경우
      result = pd.concat([result, candidate]) #후보군과 먼저 합친다.
      result = result.drop_duplicates(['title'], keep='first') #다시 중복을 제거하고
      result = result[:top_n] #열개만큼 잘라낸 뒤,
      result = result.sort_values('weighted_vote',ascending=False) #다시 별점 순으로 변경
    return result

In [228]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '어벤져스: 엔드게임', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
          title genre                             summary_processed
326  어벤져스: 엔드게임    액션  운명 펼쳐지 이후 위대 살아남 지구 떠나 위하 전쟁 절반 마지막 바꾸 최후 희망


,title,vote,weighted_vote,genre,summary_processed
0,토이 스토리 4,9.23,9.151045,애니메이션,운명 한편 거부 통하 새롭 모험 떠나 친구 위하 나서 여행 장난감 만나 세상 버즈 ...
1,어벤져스: 인피니티 워,9.08,9.067384,액션,운명 펼쳐지 목격 걸리 클라이맥스 최강 무한 대결 이루 향하 마블 스톤 조합 세계 역대
3,다크 나이트,9.65,8.617197,액션,운명 기사 조직 결전 정체 전체 대결 매이 죽이 혼돈 펼치 부패 범죄 시작 내던지 ...
5,존 윅 3: 파라벨룸,8.64,8.604717,액션,직면 킬러 파문 피아 내려지 전쟁 평화 국제 조치 찾아가 보호 세계 향하 마지막 결...
6,아일라,9.13,8.586829,드라마,운명 서로 잠시 무엇 이름 전쟁 터키어 참전 돌아가 발견 고국 그리움 모두 파병 지...
7,"안녕, 티라노: 영원히, 함께",8.6,8.511366,애니메이션,이제 혼자 펼쳐지 우정 비밀 관하 사랑 약속 이야기 떠나 희망 니야 가슴 아프 간직 천국
8,캡틴 마블,8.45,8.450464,액션,이루 전쟁 사무엘 발견 요원 지구 캐럴 크리 버스 향하 시절 희망 빠지 살아가 전사...
9,안나,8.2,8.402304,액션,제거 루스 액션 시작 살아남 파리 미션 킬러 위하 모델 위장 하드 모든 코어 위협 마지막


In [229]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '보헤미안 랩소디', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
       title genre                                  summary_processed
38  보헤미안 랩소디   드라마  아웃사이더 성장 퍼포먼스 이민자 독창적 들어가 데뷔 멤버 이름 결별 사로잡 시대 흔...


,title,vote,weighted_vote,genre,summary_processed
0,스파이더맨: 뉴 유니버스,9.38,9.247214,애니메이션,싸우 이후 피터 거미 누아르 모든 시대 구하 유니 시작 버스 있음 존재 악당 마주치...
1,너의 이름은.,9.02,9.004857,애니메이션,운명 서로 친구 가족 깨닫 되어가 이름 확인 시작 다가오 도쿄 낯설 너르 남기 풍경...
2,생일,9.01,8.902019,드라마,영원 서로 이후 올해 친구 가족 세상 그리움 모두 남기 아들 선물 특별 이야기 떠나...
3,위로공단,9.02,8.657658,다큐멘터리,단지 신음 눈물 독창적 빌딩 노동 가족 이름 과거 기나길 동의 다큐멘터리 나이키 스...
4,아일라,9.13,8.586829,드라마,운명 서로 잠시 무엇 이름 전쟁 터키어 참전 돌아가 발견 고국 그리움 모두 파병 지...
6,로켓맨,8.8,8.585797,드라마,알아보 퍼포먼스 대중 자리잡 환상적 친구 상처 사로잡 시대 세상 음악성 최고 천재적...
8,쉰들러 리스트,9.3,8.580307,드라마,리스트 의하 주의 강제 수단 노동 만들 참혹 시대 흔들리 방법 구하 학살 적히 명단...
9,모리스,8.8,8.508101,드라마,생활 서로 우정 발전 흔들리 변하 대학교 시작 이브 사람 해방감 가깝 존재 사랑 버...


In [230]:
similar_movies = find_sim_movie(movie, summ_sim_sorted_ind, '알라딘', 10)
similar_movies[['title','vote','weighted_vote','genre','summary_processed']]

추천 기준 영화 정보
   title genre                                  summary_processed
50   알라딘    모험  휘말리 시대 생각 자파 머나멀 모험 램프 의뢰 소원 마음 마법 들어주 사막 나서 신...


,title,vote,weighted_vote,genre,summary_processed
0,토이 스토리 4,9.23,9.151045,애니메이션,운명 한편 거부 통하 새롭 모험 떠나 친구 위하 나서 여행 장난감 만나 세상 버즈 ...
1,이웃집 토토로,9.52,8.941242,애니메이션,친구 언니 취하 도착 토토 생명체 모험 아빠 위태 소식 특별 도시 어머니 메이 연락...
2,레드슈즈,9.14,8.938324,애니메이션,영원 운명 사고 레드 생각 끊이 초록색 구두 저주 시작 사건 아빠 꿈꾸 난쟁이 한편...
3,겨울왕국 2,8.95,8.926177,애니메이션,운명 트롤 알려주 과거 목소리 구하 평화 시작 스토프 모험 조언 크리 부르 두려워하...
4,왕이 될 아이,9.1,8.639181,모험,학교 어로 레베카 루이스 잠시 미래 친구 모으 구하 발견 세상 절대악 스튜 엑스 패...
5,라이온 킹,8.64,8.622503,모험,모습 삼촌 도움 친구 과거 시대 왕인 세상 의욕 죄책감 되찾 마주하 사자 도전 욕망...
6,시인 할매,9.53,8.558421,다큐멘터리,박사 세월 소원 석자 배우 마음 위로 한편 서럽 견디 만이 가족 이름 만나 시집살이...
7,필그리미지,7.8,8.454642,모험,교황 전쟁 노리 수도 도중 일행 지키 사가 나타나 향하 랜드 원하 위하 처하 승리 ...
8,신비한 동물들과 그린델왈드의 범죄,7.46,7.595966,모험,탈출 운명 요청 대결 레드 도움 메인 장담 사용 모으 사회 야욕 활약 순혈 알버스 ...
